In [ ]:
# set package parent path
import os
import sys

def get_parent_path(parent_folder_name: str = "src"):
    cur_path = os.path.abspath('')
    parent_path = cur_path[:cur_path.find(parent_folder_name) + len(parent_folder_name)]

    return parent_path   

parent_path = get_parent_path()
sys.path.append(parent_path)

In [ ]:
from transformers import (AutoModelForTokenClassification,
                          DataCollatorForTokenClassification,
                          EarlyStoppingCallback, IntervalStrategy,
                          TrainingArguments)

from data.preprocessing.base_model_dataset.multitask_dataset import \
    MultitaskBioDataset
from data.preprocessing.task_dataset.ner_dataset import NerBioDataset
from data.preprocessing.task_dataset.pos_dataset import PosBioDataset
from data.preprocessing.task_dataset.srl_dataset import SrlBioDataset
from data.preprocessing.task_dataset.task_dataset import DatasetSize
from models.base.multitask import MultitaskModel
from models.config import MultitaskConfig, TaskConfig
from utils.metric.srl import compute_metrics
from utils.trainer import BaseModelArgs, MultitaskTrainer, setup_before_train
from utils.visualize import ModelVisualize

# Setup

In [ ]:
# ----------------------------------------------------------------------------------
srl_dataset = SrlBioDataset(
    model_name=args.encoder_model_name,
    data_dir=args.data_file_path,
    label_file_name=args.label_file_path
)
ner_dataset = NerBioDataset(
    model_name=args.encoder_model_name,
    data_dir=f"{args.parent_path}/data/processed/sm_bio_ner.csv",
    label_file_name=f"{args.parent_path}/data/processed/sm_bio_ner_label.csv"
)
# pos_dataset = PosBioDataset(model_name=model_name)

srl_label_vectorizer = srl_dataset.get_label_vectorizer(args.label_file_path)
ner_label_vectorizer = ner_dataset.get_label_vectorizer(f"{args.parent_path}/data/processed/sm_bio_ner_label.csv")
# pos_label_vectorizer = ner_dataset.get_label_vectorizer("./data/processed/bio_pos_label.csv")

srl_config = TaskConfig.create(
    num_labels=len(srl_label_vectorizer.label_vocab), 
    id2label=srl_label_vectorizer.label_vocab.rev_vocab,
    label2id=srl_label_vectorizer.label_vocab.vocab
)

ner_config = TaskConfig.create(
    num_labels=len(ner_label_vectorizer.label_vocab),
    id2label=ner_label_vectorizer.label_vocab.rev_vocab,
    label2id=ner_label_vectorizer.label_vocab.vocab
)

# pos_config = AutoConfig.from_pretrained(
#     model_name, 
#     num_labels=len(pos_label_vectorizer.label_vocab), 
#     id2label=pos_label_vectorizer.label_vocab.rev_vocab, 
#     label2id=pos_label_vectorizer.label_vocab.vocab
# )

config = MultitaskConfig(
    encoder_model_name=args.encoder_model_name,
    task_configs={
        "srl": srl_config,  
        "ner": ner_config,
        # "pos": pos_config
    }
)

model = MultitaskModel(config=config)


# ----------------------------------------------------------------------------------
task_datasets = {
    "srl": srl_dataset.from_cache(args.data_file_path),
    "ner": ner_dataset.from_cache(f"{args.parent_path}/data/processed/sm_bio_ner.csv"),
    # "pos": pos_dataset.create_transformer_dataset("./data/processed/bio_pos.csv")
}

for task_name, task_dataset in task_datasets.items():
    task_dataset.init_label_vectorizer()

dataset = MultitaskBioDataset(task_datasets=task_datasets)

data_collator = DataCollatorForTokenClassification(
    tokenizer=dataset.tokenizer, padding=True
)

def compute_metrics_callback(p):
    return compute_metrics(p, srl_label_vectorizer.label_vocab)

training_args = TrainingArguments(
    output_dir=args.model_save_dir,
    logging_dir=args.logging_dir,
    learning_rate=args.lr,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=args.batch_size,
    per_device_eval_batch_size=args.batch_size,
    num_train_epochs=args.epochs,
    logging_steps=args.eval_steps,
    eval_steps=args.eval_steps,
    save_steps=args.eval_steps,
    evaluation_strategy=IntervalStrategy.STEPS,
    load_best_model_at_end=True,
    metric_for_best_model="eval_srl_f1",
    push_to_hub=True,
    hub_private_repo=True,
    weight_decay=0.01,
    max_steps=4500
)

trainer = MultitaskTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset.get_train_datasets(),
    eval_dataset=dataset.get_valid_datasets(),
    compute_metrics=compute_metrics_callback,
    data_collator=data_collator,
    tokenizer=dataset.tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

## Train

In [ ]:
output = trainer.train()